In [2]:
import import_ipynb
from libraries import *;
from gates import *;
from QSD import *;
from hamiltonian import *;

importing Jupyter notebook from libraries.ipynb
importing Jupyter notebook from gates.ipynb
importing Jupyter notebook from QSD.ipynb
importing Jupyter notebook from gray.ipynb
importing Jupyter notebook from md.ipynb
importing Jupyter notebook from hamiltonian.ipynb


In [3]:
# Noiseless simulated backend
sim = QasmSimulator()

In [4]:
# Parameterize variable t to be evaluated at t=pi later
t = Parameter('t')
num_qubits = 3

In [5]:
def reveal(instructions, num_qubits):
    """
    Given circuit instructions and size, prints drawing of circuit and circuit parameters
    """
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    qc.append(instructions, qr)
    qc = qc.decompose().decompose()
    print(qc.decompose().draw())
    print(qc.decompose().count_ops())
    return
    

In [6]:
def line(k):
     return [[i, i+1] for i in range(k - 1) ] + [[i + 1, i] for i in range(k - 1) ]

In [55]:
def basic_trotter(terms = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], time = np.pi, num_qubits = 3, qsd = False, name = "Trot"):
    """ 
    Constructs basic trotter decomposition for \exp[i H_heis t]
    Inputs: terms, list of strings, indicating gate sequence
            time, float
    Returns: QuantumCircuit instructions
    expected number of cxs is 12
    """
    
    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name=name)

    for index in terms:
        j, i = index[0], int(index[1]) - 1
        Trot_qc.append(QQ_gate(j, time = time), [Trot_qr[i], Trot_qr[i+1]])
        
    if qsd:
        Trot_qc = gen_qsd(Operator(Trot_qc).data)
        
    # Convert custom quantum circuit into a gate
    Trot_gate = Trot_qc.to_instruction()
    
    return Trot_qc

In [51]:
def ti_trotter(terms = None, time=np.pi, qsd = False):
    Trot_qr = QuantumRegister(num_qubits)
    Trot_qc = QuantumCircuit(Trot_qr, name='Trot')
    
    ti_terms = ['x1', 'y1', 'z1']
    ti_evolution = basic_trotter(ti_terms, time, num_qubits = 2, qsd = qsd)
    Trot_qc.append(ti_evolution, [Trot_qr[0], Trot_qr[1]])
    Trot_qc.append(ti_evolution, [Trot_qr[1], Trot_qr[2]])
    
    # Convert custom quantum circuit into a gate
    Trot_gate = Trot_qc.decompose()
    Trot_gate = Trot_qc.to_instruction()
    return Trot_qc

In [52]:
def basic_trotter_circuit(trotter_gate_function, trotter_steps, num_qubits = 7, time = np.pi, terms = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], operational_qubits = [0, 1, 2], qsd = False):
    """ 
    Constructs Trotter Circuit for Jakarta machine by Repeating a given gate
    Inputs: trotter_gate_function, function, returns QuantumCircuit instructions 
            trotter_steps, int
            target_time, final simulation time, assumed = np.pi
            terms, list of strings, indicating gate sequence
    
    Returns: QuantumRegister, QuantumCircuit 
    """
    q1, q2, q3 = operational_qubits
        
    # Initialize quantum circuit for n qubits
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    
    # Simulate time evolution under H_heis3 Hamiltonian
    for _ in range(trotter_steps):
        qc.append(trotter_gate_function(terms = terms,time=time/trotter_steps, qsd = qsd), [qr[q1], qr[q2], qr[q3]])
        
    # Evaluate simulation at target_time (t=pi) meaning each trotter step evolves pi/trotter_steps in time
    qc = qc.decompose()
    
    return qr, qc 

In [53]:
def second_order_trotter(trotter_steps, num_qubits = 7, time = np.pi, operational_qubits = [0, 1, 2],  qsd = False):
    q1, q2, q3 = operational_qubits
        
    # Initialize quantum circuit for n qubits
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    
    if trotter_steps == 1:
        hot = ti_higher_order_trotter(order = 2, time = t, terms = None)
        instr = hot(time = time, qsd = False)
        qc.append(instr, [qr[q1], qr[q2], qr[q3]])
        return qr, qc
    
    # basic gates
    ttime = time / trotter_steps
    ti_terms = ['x1', 'y1', 'z1']
    ti_half_evolution = basic_trotter(ti_terms, ttime / 2, num_qubits = 2, qsd = qsd)
    ti_evolution = basic_trotter(ti_terms, ttime, num_qubits = 2, qsd = qsd)
     
    # circuit assembly
    qc.append(ti_half_evolution, [qr[q1], qr[q2]])

    for _ in range(trotter_steps - 1):
        qc.append(ti_evolution, [qr[q2], qr[q3]])
        qc.append(ti_evolution, [qr[q1], qr[q2]])
        
    qc.append(ti_evolution, [qr[q2], qr[q3]])
    qc.append(ti_half_evolution, [qr[q1], qr[q2]])
    
    return qr, qc   

In [27]:
def ti_higher_order_trotter(order = 2, time = np.pi, terms = None):
    def wrapper(terms = terms, time = time, qsd = False):
        
        num_qubits = 3
        Trot_qr = QuantumRegister(num_qubits)
        Trot_qc = QuantumCircuit(Trot_qr, name='Higher_Order_Trot')
        ti_terms = ['x1', 'y1', 'z1']

        coefficients = higher_trotter_coefficients(['1','2'], order, time)

        for term, coefficient in coefficients:
            i = int(term) - 1
            ti_evolution = basic_trotter(ti_terms, time = coefficient, num_qubits = 2, qsd = qsd)
            Trot_qc.append(ti_evolution, [Trot_qr[i], Trot_qr[i+1]])
            
        # Convert custom quantum circuit into a gate
        Trot_gate = Trot_qc.to_instruction()
        return Trot_gate
    
    return wrapper

In [26]:
def higher_order_trotter(order = 2, time = np.pi, terms = ['x1', 'y1', 'z1', 'x2', 'y2', 'z2']):
    '''
    Generates higher order trotter gate function.
    Inputs: order, int, order of higher order formula
            time, float, time of simulation of this trotter step
            random, boolean, random choice of term ordering?
            terms, list of strings, indicating order of decomposition
    Returns: Function, which returns QuantumCircuit instructions
    '''
    
    def wrapper(terms = terms, time = time, qsd = False):

        num_qubits = 3
        Trot_qr = QuantumRegister(num_qubits)
        Trot_qc = QuantumCircuit(Trot_qr, name='TI Higher_Order_Trot')

        coefficients = higher_trotter_coefficients(terms, order, time)

        for term, coefficient in coefficients:
            j, i = term[0], int(term[1]) - 1
            Trot_qc.append(QQ_gate(j, coefficient), [Trot_qr[i], Trot_qr[i+1]])
            
        # Convert custom quantum circuit into a gate
        Trot_gate = Trot_qc.to_instruction()
        return Trot_gate
    
    
    return wrapper

In [23]:
def higher_trotter_coefficients(terms, order, time):
    """ 
    Recursively computes the order and the exponents of the higher order trotter decomposition
    Implementation based on the paper The Theory of Trotter Error [need citation]
    Inputs: terms, list of strings, indicating gate sequence
            order, int, order of formula
            time, float
    Returns: list of tuples, (string, float)
    """
    
    coefficients = []
    
    if order == 1:
        for term in terms:
            coefficients.append((term, time))
        return coefficients
        
    if order == 2:
        for term in terms[:-1]:
            coefficients.append((term, time/2))
        
        coefficients.append((terms[-1], time))
        
        for term in reversed(terms[:-1]):
            coefficients.append((term, time/2))
            
        return coefficients
    
    u_order = 1/(4 - 4**(1/(order-1)))
    
    previous = higher_trotter_coefficients(terms, order - 2, u_order*time)
    middle = higher_trotter_coefficients(terms, order - 2, (1 - 4*u_order)*time)
    
    coefficients = previous + previous + middle + previous + previous
    
    return hot_parse_coefficients(coefficients)
    

In [24]:
def hot_parse_coefficients(coefficients):
    
    new_c = []
    current_term, current_coefficient = coefficients[0] 
     
    for term, coef in coefficients[1:]:
        if term != current_term:
            new_c.append([current_term, current_coefficient])
            current_term = term
            current_coefficient = coef
        else:
            current_coefficient += coef
            
    new_c.append([current_term, current_coefficient])
    return new_c
    
        

In [25]:
def qsd_qc(t):
    
    time_evol = U_heis3(t).to_matrix()
    circuit = gen_qsd(time_evol).decompose()
    
    return circuit.decompose().to_instruction()

In [149]:
def single_layer(time, num_qubits, qsd = True):
    
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    
    ti_terms = ['x1', 'y1', 'z1']
    ti_evolution = basic_trotter(ti_terms, time, num_qubits = 2, qsd = qsd, name = "Brick")
    
    # even terms
    for i in range(0, num_qubits - 1, 2):
        qc.append(ti_evolution, [i, i+1])
        
    # odd terms
    for i in range(1, num_qubits - 1, 2):
        qc.append(ti_evolution, [i, i+1])
    
    return qc

In [150]:
def brickwork(time, num_qubits, trotter_steps = 1, qsd = True):
    
    qr = QuantumRegister(num_qubits)
    qc = QuantumCircuit(qr)
    
    layer = single_layer(time/trotter_steps, num_qubits, qsd = qsd)
    
    for _ in range(trotter_steps):
        qc.append(layer, qr)
        
    qc = qc.decompose()
        
    return qr, qc